# (4~10)년도 예측

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import copy
import json
import random
import io
import distutils.dir_util
import warnings
warnings.filterwarnings("ignore")

from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity

import math
import os, sys, gc, time, warnings, pickle, psutil, random
import gc
import time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
from collections import Counter
from tqdm import tqdm



In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.models import load_model
from keras.initializers import glorot_normal, Zeros, Ones
import keras.backend as K
from keras.optimizers import RMSprop
import tensorflow as tf

## make a model
from keras.layers import Input, Dense
from keras.models import Model


In [ ]:
train = pd.read_parquet('/kaggle/input/song-traim-song-tag-dict-tag-song-dict/train(song_trim).parquet')


In [ ]:

# small_years = [11, 12, 13]
# file_path = '/kaggle/input/yearly-pv-file-0-10-11-13-14-15/train_DF_%s.parquet' %small_years
# train_pv = pd.read_parquet(file_path)

small_years = [i for i in range(4,11)]
train_pv = pd.read_parquet('/kaggle/input/yearly-pv-file-0-10-11-13-14-15/train_DF_[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10].parquet')


In [ ]:
print(train_pv.shape)
display(train_pv.head(2))
display(train_pv.tail(2))

(21630, 5719)


id,49,59,92,104,121,129,204,208,252,290,...,153190,153218,153221,153229,153230,153258,153313,153333,153392,153426
song,,,,,,,,,,,,,,,,,,,,,
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


id,49,59,92,104,121,129,204,208,252,290,...,153190,153218,153221,153229,153230,153258,153313,153333,153392,153426
song,,,,,,,,,,,,,,,,,,,,,
707896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
707960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## autoencoder

In [ ]:
input_dim = train_pv.shape[1]
print(input_dim)

c=  500

model = Sequential()

model.add(Dense(c, input_dim=input_dim, activation='relu'))
model.add(Dense(input_dim, activation='sigmoid'))

model.compile(optimizer='adam', loss='mse') # (optimizer='adadelta', loss='binary_crossentropy')

5719


In [ ]:
## train
tmp = train_pv.values
history = model.fit(tmp, tmp,
                epochs=4,
                batch_size=256,
                shuffle=True)

Epoch 1/4
85/85 [==============================] - 8s 90ms/step - loss: 0.0978
Epoch 2/4
85/85 [==============================] - 8s 97ms/step - loss: 0.0022
Epoch 3/4
85/85 [==============================] - 8s 91ms/step - loss: 0.0014
Epoch 4/4
85/85 [==============================] - 8s 90ms/step - loss: 0.0012


In [ ]:
model.summary()
encoder = Model(model.input, model.layers[0].output)
encoded_imgs = encoder.predict(tmp)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               2860000   
_________________________________________________________________
dense_1 (Dense)              (None, 5719)              2865219   
Total params: 5,725,219
Trainable params: 5,725,219
Non-trainable params: 0
_________________________________________________________________


In [ ]:
encoder.save("encoder%s" %small_years)


In [ ]:
del tmp, model, encoder
gc.collect()

2709

In [ ]:
train_data = pd.DataFrame(data=encoded_imgs)

del encoded_imgs
gc.collect()

20

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_array = cosine_similarity(train_data, train_data)
print('cosine_array complete')
# item_based_array= pd.DataFrame(data = cosine_array, index = train_pv.index, columns = train_pv.index)


index2id = {i:u for i, u in enumerate((train_pv.index))}

cf_dict = {}
for i in tqdm(range(len(cosine_array))):
    song_id= list(map(index2id.get, list(np.argsort(-cosine_array[i])[1:201])))
    value= list(np.sort(cosine_array[0])[::-1][1:201])
    cf_dict[index2id[i]] = list(zip(song_id,value))

del cosine_array, train_data
gc.collect()

  0%|          | 30/21630 [00:00<01:14, 290.86it/s]

cosine_array complete


100%|██████████| 21630/21630 [01:15<00:00, 285.77it/s]


0

## 본격 Predict

In [ ]:
def most_popular(playlists, col, topk_count):
    c = Counter()

    for doc in playlists[col]:
        c.update(doc)

    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

def remove_seen(seen, l):
    seen = set(seen)
    ## 데이터에 l의 원소가 없으면 l의 원소를 채워 넣는다. 
    return [x for x in l if not (x in seen)]

In [ ]:
val = pd.read_json('/kaggle/input/song-traim-song-tag-dict-tag-song-dict/test(lower).json')


In [ ]:
def check_target_type(val):
    no_song = []
    no_both = []
    no_tag = []
    yes_index = []

    for i in range(len(val)):
        if (not val.loc[i:i, 'songs'].values[0])&(not not val.loc[i:i, 'tags'].values[0]): ## 노래는 없는데 태그는 있어
            no_song.append(i)

        elif (not not val.loc[i:i, 'songs'].values[0]) & (not val.loc[i:i, 'tags'].values[0]): ## 노래는 있는데 태그가 없으
            no_tag.append(i)  

        elif (not val.loc[i:i, 'songs'].values[0]) & (not val.loc[i:i, 'tags'].values[0]): ## 노래도 없고 태그도 없어
            no_both.append(i)          

        else: ## 둘다 있음, 아마도 노래 몇 곡이 누락 됐어 
            yes_index.append(i)
            
    print("노래가 하나도 없음 : ", len(no_song))
    print("노래, 태그 둘 다 존재함 : ", len(yes_index))
    print("태그가 없음 : ",len(no_tag) )
    print("둘다 없음 : ", len(no_both))
    
    return no_song, no_both, no_tag, yes_index
        

In [ ]:

no_song, no_both, no_tag, yes_index = check_target_type(val)


노래가 하나도 없음 :  1232
노래, 태그 둘 다 존재함 :  4190
태그가 없음 :  4507
둘다 없음 :  811


## 3. 둘 다 있음
- 기존에 존재하던 tag에서 가장 빈번하게 나타났던 음악을 넣어주기 

In [ ]:
v1v3_index= yes_index+ no_tag

In [ ]:
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))
train['date'] = train['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )

val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))
val['date'] = val['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )


year_list = list(train['year'].unique())
year_list= sorted(year_list)

v1v3 = val[val.index.isin(v1v3_index)]


val_tmp = v1v3[v1v3['year'].isin(small_years)]
print("val year shape", val_tmp.shape)
val_tmp.reset_index(inplace = True, drop = True)

val year shape (484, 8)


## popular monthly tag

In [ ]:


date_list = list(train['date'].unique())
date_list= sorted(date_list)

date_dict = {}
for i,k in enumerate(date_list):
    date_dict[k] = i

popular_date_tag = []
for i in range(len(date_list)):
    if i <= 13:  ## 13이전에는 한 달에 100개가 안나옴
        year = int(str(date_list[i])[0]) ## year
        tmp = train[train['year'] == year]
        _, year_tag = most_popular(tmp, "tags", 100)
        popular_date_tag.append({
            "date" : date_list[i],
            "tags" : year_tag,})
        
    else:
        tmp = train[train['date'] == date_list[i]]
        _, date_tag = most_popular(tmp, "tags", 100)
        popular_date_tag.append({
            "date" : date_list[i],
            "tags" : date_tag,})  

## popular monthly song

In [ ]:
popular_date_song = []
for i in range(len(date_list)):
    if i <= 9:  ## 11이전에는 한 달에 100개가 안나옴
        year = int(str(date_list[i])[0]) ## year
        tmp = train[train['year'] == year]
        _, date_song = most_popular(tmp, "songs", 200)
        popular_date_song.append({
            "date" : date_list[i],
            "songs" : date_song,})               
        
    else:
        tmp = train[train['date'] == date_list[i]]
        _, date_song = most_popular(tmp, "songs", 200)
        popular_date_song.append({
            "date" : date_list[i],
            "songs" : date_song,})  

In [ ]:
song_tag_dict = load_json('/kaggle/input/song-traim-song-tag-dict-tag-song-dict/song_tag_dict.json')


In [ ]:
def f7(seq):
    '''
    list에 있는 중복 데이터는 삭제하고, 순서는 유지하는 함수
    set만 사용하면 순서가 뒤엉키게 됨
    '''
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [ ]:
v1v3_predict = []
for i in range(len(val_tmp)):
    date = date_dict[val_tmp.loc[i:i, 'date'].values[0]]
    
    
    ################ song 채우기 ###############################
    songs = val_tmp.loc[i:i, 'songs'].values[0]
    songs = [x for x in songs if x in cf_dict]

    if len(songs):
        tmp_list = []
        for song in songs:
            tmp_list.extend(cf_dict[song]) 
        tmp_list.sort(key = lambda x: x[1], reverse = True)    
        tmp_list = [i[0] for i in tmp_list if i[1]>0]
        tmp_list = f7(tmp_list)

        cur_song = tmp_list[:200]
    else:
        cur_song = popular_date_song[date]['songs']

    cur_song = remove_seen(list(val_tmp.loc[i:i, 'songs'])[0], cur_song)[:100]

    #중복 케이스 때문에 부족하면 popular song으로 다시 채워줌
    if len(cur_song) != 100:
        update_song = remove_seen(cur_song, popular_date_song[date]['songs'])
        cur_song.extend(update_song)
        cur_song = cur_song[:200]
    
    
    ############### tag 채우기 ##################################
    tag_counter = Counter()
    for song in list(val_tmp.loc[i:i, 'songs'].values[0]):
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    tag_counter = tag_counter[:100] ## 넉넉히 100개 잡는다. 
    
    cur_tag = []
    for k in tag_counter:
        cur_tag.append(k[0])     
        
    if (len(cur_tag) < 100) & (len(cur_tag) > 0 ):
        ## 부족하면 채워준다.
        update_tag = remove_seen(cur_tag, popular_date_tag[date]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:100]
    if len(cur_tag) == 0:
        cur_tag = popular_date_tag[date]['tags'][:100]
        
    #############################################################    

    v1v3_predict.append({
        "id" : list(val_tmp.loc[i:i, 'id'])[0],
        "songs": remove_seen(list(val_tmp.loc[i:i, 'songs'])[0], cur_song)[:100],
        "tags": remove_seen(list(val_tmp.loc[i:i, 'tags'])[0], cur_tag)[:10],
     })    

    


In [ ]:

write_json(v1v3_predict, 'v1v3_%s_predict.json' %small_years)

In [ ]:
# from IPython.display import FileLink
# FileLink(r'v1v3_%s_predict.json' %small_years)